ПИ18-1 Гриднев Д.В.
Номер 133
Вариант 3.
Построение модели и оптимизация гиперпараметров. Датасет: https://www.kaggle.com/shebrahimi/financial-distress

In [1]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn import ensemble
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use("seaborn")

In [2]:
df = pd.read_csv("./fd.csv")
df.head()

,Company,Time,Financial Distress,x1,x2,x3,x4,x5,x6,x7,...,x74,x75,x76,x77,x78,x79,x80,x81,x82,x83
0,1,1,0.010636,1.2810,0.022934,0.87454,1.21640,0.060940,0.188270,0.52510,...,85.437,27.07,26.102,16.000,16.0,0.2,22,0.060390,30,49
1,1,2,-0.455970,1.2700,0.006454,0.82067,1.00490,-0.014080,0.181040,0.62288,...,107.090,31.31,30.194,17.000,16.0,0.4,22,0.010636,31,50
2,1,3,-0.325390,1.0529,-0.059379,0.92242,0.72926,0.020476,0.044865,0.43292,...,120.870,36.07,35.273,17.000,15.0,-0.2,22,-0.455970,32,51
3,1,4,-0.566570,1.1131,-0.015229,0.85888,0.80974,0.076037,0.091033,0.67546,...,54.806,39.80,38.377,17.167,16.0,5.6,22,-0.325390,33,52
4,2,1,1.357300,1.0623,0.107020,0.81460,0.83593,0.199960,0.047800,0.74200,...,85.437,27.07,26.102,16.000,16.0,0.2,29,1.251000,7,27


Переменные чистить не трубуется

In [3]:
df.isna().sum()

Company               0
Time                  0
Financial Distress    0
x1                    0
x2                    0
                     ..
x79                   0
x80                   0
x81                   0
x82                   0
x83                   0
Length: 86, dtype: int64

Разбиваю датасест

In [4]:
X = df[df.columns.difference(["Financial Distress"])]
Y = df["Financial Distress"]

In [5]:
X_train, X_test, y_train, y_test = train_test_split(
    X, Y, test_size=0.33, random_state=42
)

1. Построение модели
Выбрал Random Forest, так как он в большенстве случаев дает лучшие результаты

In [6]:
reg = RandomForestRegressor(criterion='mae', max_depth=12, max_features='auto', n_estimators=150, n_jobs=-1)
reg.fit(X_train, y_train)

RandomForestRegressor(criterion='mae', max_depth=12, n_estimators=150,
                      n_jobs=-1)

In [7]:
reg.score(X_train, y_train)

0.9170690788286889

In [8]:
reg.score(X_test, y_test)

0.14162702704063435

2. Оптимизация гиперпараметров

Обучение очень долгое, не могу сделать больше гиперпараметров

In [9]:
reg = RandomForestRegressor()
param_grid = {
    "n_estimators": [50, 100],
    "criterion": ["mse", "mae"],
    "max_depth": [2, 3],
    "max_features": ["sqrt", "log2"],
}

In [10]:
GSCV_reg = GridSearchCV(reg, param_grid, n_jobs=-1, cv=3, verbose=1)
GSCV_reg.fit(X_train, y_train)

Fitting 3 folds for each of 16 candidates, totalling 48 fits


GridSearchCV(cv=3, estimator=RandomForestRegressor(), n_jobs=-1,
             param_grid={'criterion': ['mse', 'mae'], 'max_depth': [2, 3],
                         'max_features': ['sqrt', 'log2'],
                         'n_estimators': [50, 100]},
             verbose=1)

In [14]:
GSCV_reg.best_params_

{'criterion': 'mse',
 'max_depth': 3,
 'max_features': 'sqrt',
 'n_estimators': 100}

Проверка 

In [31]:
reg = RandomForestRegressor(criterion='mse', max_depth=3, max_features='sqrt', n_estimators=100, n_jobs=-1)
reg.fit(X_train, y_train)

RandomForestRegressor(max_depth=3, max_features='sqrt', n_jobs=-1)

In [32]:
reg.score(X_train, y_train)

0.537294009147689

In [33]:
reg.score(X_test, y_test)

0.09247844200111432

Скор упал, так как max_depth стал хуже. Ниже пример где в grid search вставляется правильный max_depth

3. Модель использует CatBoostRegressor, на оптимизацию гиперпараметров не хватил времени

In [20]:
from catboost import CatBoostRegressor

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42)

model = CatBoostRegressor(iterations=666,
                          learning_rate=1,
                          depth=1)
model.fit(X_train, y_train)

0:	learn: 1.3026846	total: 1.76ms	remaining: 1.17s
1:	learn: 1.2361583	total: 3.87ms	remaining: 1.28s
2:	learn: 1.2109847	total: 5.39ms	remaining: 1.19s
3:	learn: 1.2004561	total: 6.73ms	remaining: 1.11s
4:	learn: 1.1814547	total: 8.59ms	remaining: 1.14s
5:	learn: 1.1583245	total: 10.2ms	remaining: 1.12s
6:	learn: 1.1431603	total: 10.9ms	remaining: 1.02s
7:	learn: 1.1332359	total: 12.4ms	remaining: 1.02s
8:	learn: 1.1199246	total: 13.4ms	remaining: 977ms
9:	learn: 1.1136475	total: 14.4ms	remaining: 944ms
10:	learn: 1.1069488	total: 15.3ms	remaining: 913ms
11:	learn: 1.0998055	total: 16.2ms	remaining: 883ms
12:	learn: 1.0933293	total: 17.4ms	remaining: 872ms
13:	learn: 1.0833985	total: 18.8ms	remaining: 874ms
14:	learn: 1.0762936	total: 20.2ms	remaining: 875ms
15:	learn: 1.0680696	total: 21.5ms	remaining: 872ms
16:	learn: 1.0639185	total: 22.2ms	remaining: 847ms
17:	learn: 1.0593151	total: 23.3ms	remaining: 839ms
18:	learn: 1.0502133	total: 24.3ms	remaining: 829ms
19:	learn: 1.0431589	t

209:	learn: 0.8468097	total: 180ms	remaining: 390ms
210:	learn: 0.8462847	total: 181ms	remaining: 390ms
211:	learn: 0.8457639	total: 182ms	remaining: 390ms
212:	learn: 0.8453533	total: 183ms	remaining: 390ms
213:	learn: 0.8448871	total: 185ms	remaining: 390ms
214:	learn: 0.8445275	total: 186ms	remaining: 389ms
215:	learn: 0.8442283	total: 187ms	remaining: 389ms
216:	learn: 0.8438291	total: 188ms	remaining: 388ms
217:	learn: 0.8435247	total: 189ms	remaining: 389ms
218:	learn: 0.8433043	total: 191ms	remaining: 390ms
219:	learn: 0.8429599	total: 192ms	remaining: 390ms
220:	learn: 0.8424269	total: 194ms	remaining: 390ms
221:	learn: 0.8421236	total: 195ms	remaining: 390ms
222:	learn: 0.8416942	total: 196ms	remaining: 390ms
223:	learn: 0.8412692	total: 198ms	remaining: 390ms
224:	learn: 0.8405419	total: 199ms	remaining: 390ms
225:	learn: 0.8400223	total: 200ms	remaining: 390ms
226:	learn: 0.8395274	total: 202ms	remaining: 391ms
227:	learn: 0.8390257	total: 204ms	remaining: 391ms
228:	learn: 

502:	learn: 0.7496021	total: 515ms	remaining: 167ms
503:	learn: 0.7494557	total: 517ms	remaining: 166ms
504:	learn: 0.7492165	total: 518ms	remaining: 165ms
505:	learn: 0.7489990	total: 519ms	remaining: 164ms
506:	learn: 0.7487311	total: 521ms	remaining: 163ms
507:	learn: 0.7484561	total: 522ms	remaining: 162ms
508:	learn: 0.7482388	total: 523ms	remaining: 161ms
509:	learn: 0.7480452	total: 525ms	remaining: 161ms
510:	learn: 0.7478372	total: 526ms	remaining: 160ms
511:	learn: 0.7476365	total: 527ms	remaining: 159ms
512:	learn: 0.7474457	total: 529ms	remaining: 158ms
513:	learn: 0.7472453	total: 530ms	remaining: 157ms
514:	learn: 0.7470623	total: 531ms	remaining: 156ms
515:	learn: 0.7468934	total: 532ms	remaining: 155ms
516:	learn: 0.7466886	total: 533ms	remaining: 154ms
517:	learn: 0.7463125	total: 535ms	remaining: 153ms
518:	learn: 0.7460048	total: 536ms	remaining: 152ms
519:	learn: 0.7456521	total: 537ms	remaining: 151ms
520:	learn: 0.7453724	total: 539ms	remaining: 150ms
521:	learn: 

659:	learn: 0.7175018	total: 695ms	remaining: 6.32ms
660:	learn: 0.7173484	total: 696ms	remaining: 5.27ms
661:	learn: 0.7170796	total: 697ms	remaining: 4.21ms
662:	learn: 0.7168638	total: 698ms	remaining: 3.16ms
663:	learn: 0.7166754	total: 699ms	remaining: 2.11ms
664:	learn: 0.7164255	total: 701ms	remaining: 1.05ms
665:	learn: 0.7162800	total: 702ms	remaining: 0us


In [21]:
print("catboost train score: ", model.score(X_train, y_train))
print("catboost test score: ", model.score(X_test, y_test))

catboost train score:  0.8040075909850519
catboost test score:  0.12223445983194059


4. Я заметил лишние фичи

In [35]:
df = pd.read_csv("./fd.csv")
df = df.drop(['Company', 'Time'], axis=1)

In [36]:
X = df[df.columns.difference(["Financial Distress"])]
Y = df["Financial Distress"]

In [37]:
X_train, X_test, y_train, y_test = train_test_split(
    X, Y, test_size=0.33, random_state=42
)

In [38]:
reg = RandomForestRegressor(criterion='mae', max_depth=12, max_features='auto', n_estimators=150, n_jobs=-1)
reg.fit(X_train, y_train)

RandomForestRegressor(criterion='mae', max_depth=12, n_estimators=150,
                      n_jobs=-1)

In [39]:
reg.score(X_train, y_train)

0.9146203838418517

In [40]:
reg.score(X_test, y_test)

0.14569975677871194

Скор вырос :)

Дальше пытаюсь еще раз провести оптимизацию гиперпараметров, успеть в срок конца экзамена :)

In [68]:
reg = RandomForestRegressor()
param_grid = {
    "n_estimators": [50, 100, 150, 200],
    "criterion": ["mae"],
    "max_depth": [12, 13, 14, 15],
    "max_features": ["sqrt"],
}

In [69]:
GSCV_reg = GridSearchCV(reg, param_grid, n_jobs=-1, cv=3, verbose=1)
GSCV_reg.fit(X_train, y_train)

Fitting 3 folds for each of 16 candidates, totalling 48 fits


GridSearchCV(cv=3, estimator=RandomForestRegressor(), n_jobs=-1,
             param_grid={'criterion': ['mae'], 'max_depth': [12, 13, 14, 15],
                         'max_features': ['sqrt'],
                         'n_estimators': [50, 100, 150, 200]},
             verbose=1)

In [70]:
GSCV_reg.best_params_

{'criterion': 'mae',
 'max_depth': 15,
 'max_features': 'sqrt',
 'n_estimators': 200}

Проверка

In [71]:
reg = RandomForestRegressor(criterion='mae', max_depth=15, max_features='sqrt', n_estimators=200, n_jobs=-1)
reg.fit(X_train, y_train)

RandomForestRegressor(criterion='mae', max_depth=15, max_features='sqrt',
                      n_estimators=200, n_jobs=-1)

In [72]:
reg.score(X_train, y_train)

0.929166119051961

In [73]:
reg.score(X_test, y_test)

0.12124427929500758

Удивительно, но скор упал, предполагаю что требуются другие гиперпараметры но нет времени для их обучения